In [1]:
import shnitsel as st

Let us document how to work with a *stacked* data set, i.e. data from various trajectories is concatenated along a `frame` dimension that includes a `atrajectory` coordinate encoding the id of the trajectory that the frame data is from, but metadata for each trajectory is stored along a `trajectory` dimension.

In [2]:
from shnitsel.data.dataset_containers import MultiSeriesStacked
frames = st.read("./test_data/shnitsel/fixtures/butene_dynamic/data_new.nc")
# Make sure the data is actually the stacked format we want.
assert isinstance(frames, MultiSeriesStacked)
frames

MultiSeriesStacked(_raw_dataset=<xarray.Dataset> Size: 83MB
Dimensions:      (state: 3, frame: 36579, statecomb: 3, atom: 12, direction: 3,
                  trajectory: 156)
Coordinates: (12/19)
  * state        (state) int64 24B 1 2 3
  * frame        (frame) object 293kB MultiIndex
  * statecomb    (statecomb) object 24B MultiIndex
  * atom         (atom) int64 96B 0 1 2 3 4 5 6 7 8 9 10 11
  * direction    (direction) <U1 12B 'x' 'y' 'z'
  * trajectory   (trajectory) int32 624B 2 3 6 11 13 16 ... 310 311 313 314 316
    ...           ...
    atNums       (atom) int64 96B ...
    delta_t      (trajectory) float64 1kB ...
  * from         (statecomb) int64 24B 1 1 2
  * to           (statecomb) int64 24B 2 3 3
  * atrajectory  (frame) int64 293kB 2 2 2 2 2 2 2 ... 316 316 316 316 316 316
  * time         (frame) float64 293kB 0.0 0.5 1.0 1.5 ... 76.5 77.0 77.5 78.0
Data variables:
    energy       (state, frame) float64 878kB ...
    nacs         (statecomb, atom, direction, frame) float64 32MB ...
    atXYZ        (atom, direction, frame) float64 11MB 0.2717 0.2719 ... 1.624
    phases       (state, frame) float64 878kB ...
    e_kin        (frame) float64 293kB ...
    forces       (state, atom, direction, frame) float64 32MB ...
    dip_perm     (state, direction, frame) float64 3MB ...
    dip_trans    (statecomb, direction, frame) float64 3MB ...
Attributes:
    delta_t:                0.5
    trajectory_input_path:  test_data/shnitsel/fixtures/butene_dynamic/data.nc
    trajectory_id:          1784668298, _is_multi_trajectory=True, _basis_data=None, _layered_repr_cached=None)

If we ever want to work with a layered dataset, the shnitsel-wrapper for the dataset provides the `as_layered` property performing automatic conversion.
- A *layered* dataset has a `trajectory` dimension that allows us to pick between different datasets and within each dataset we have a `time` or `frame` dimension to identify the frame of data), 
- A *stacked* dataset in contrast has the `trajectory` dimension only for the metadata and uses a `frame` dimension across all original datasets to index individual frames. 
So, what is the difference?
- A *layered* dataset may be more conventionally useful to address data for new users, but be aware that this structure requires all data to be padded to the same length across trajectories. 
You may encounter a lot of NaN values in variables at the end of the trajectory, which consumes more memory.
- A *stacked* dataset separates the metadata information from the actual data by indexing it along a different dimension. This allows for better compression of data, because we do not need to pad trajectories of different lengths. 
  In principle, the relation is most easily understood similar to the difference between the `state` dimension, which lets us pick between different states in metadata or in observables and the `astate` coordinate, which holds the *currently* active state in a time series.

In [3]:
frames.as_layered

/home/tpadmin/git/shnitsel-tools-official/shnitsel/data/traj_combiner_methods.py:607: FutureWarning: In a future version of xarray the default value for coords will change from coords='different' to coords='minimal'. This is likely to lead to different results when multiple datasets have matching variables with overlapping values. To opt in to new defaults and get rid of these warnings now use `set_options(use_new_combine_kwarg_defaults=True) or set coords explicitly.
  layers = xr.concat(


MultiSeriesLayered(_raw_dataset=<xarray.Dataset> Size: 104MB
Dimensions:      (trajectory: 156, state: 3, time: 300, statecomb: 3, atom: 12,
                  direction: 3)
Coordinates: (12/17)
  * trajectory   (trajectory) int32 624B 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0
  * state        (state) int64 24B 1 2 3
  * time         (time) float64 2kB 0.0 0.5 1.0 1.5 ... 148.0 148.5 149.0 149.5
  * statecomb    (statecomb) object 24B MultiIndex
  * atom         (atom) int64 96B 0 1 2 3 4 5 6 7 8 9 10 11
  * direction    (direction) <U1 12B 'x' 'y' 'z'
    ...           ...
    nsteps       (trajectory) int64 1kB 301 301 301 301 301 ... 301 301 301 301
    max_ts       (trajectory) int64 1kB 301 301 301 301 301 ... 301 301 301 301
    atNames      (atom) <U1 48B 'C' 'C' 'C' 'C' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H'
    state_names  (state) <U2 24B 'S0' 'S1' 'S2'
    atNums       (atom) int64 96B 6 6 6 6 1 1 1 1 1 1 1 1
    delta_t      (trajectory) float64 1kB 0.5 0.5 0.5 0.5 ... 0.5 0.5 0.5 0.5
Data variables:
    energy       (trajectory, state, time) float64 1MB 1.606 1.141 ... nan nan
    nacs         (trajectory, statecomb, atom, direction, time) float64 40MB ...
    atXYZ        (trajectory, atom, direction, time) float64 13MB 0.2717 ... nan
    phases       (trajectory, state, time) float64 1MB -1.0 1.0 1.0 ... nan nan
    e_kin        (trajectory, time) float64 374kB 1.271 1.641 2.014 ... nan nan
    forces       (trajectory, state, atom, direction, time) float64 40MB -0.0...
    dip_perm     (trajectory, state, direction, time) float64 3MB -0.2074 ......
    dip_trans    (trajectory, statecomb, direction, time) float64 3MB 0.7541 ...
Attributes:
    delta_t:                0.5
    trajectory_input_path:  test_data/shnitsel/fixtures/butene_dynamic/data.nc
    trajectory_id:          1784668298
    is_multi_trajectory:    True, _is_multi_trajectory=True, _basis_data=[Trajectory(_raw_dataset=<xarray.Dataset> Size: 672kB
Dimensions:      (state: 3, time: 300, statecomb: 3, atom: 12, direction: 3)
Coordinates: (12/17)
  * state        (state) int64 24B 1 2 3
  * time         (time) float64 2kB 0.0 0.5 1.0 1.5 ... 148.0 148.5 149.0 149.5
  * statecomb    (statecomb) object 24B MultiIndex
  * atom         (atom) int64 96B 0 1 2 3 4 5 6 7 8 9 10 11
  * direction    (direction) <U1 12B 'x' 'y' 'z'
    sdiag        (time) int64 2kB ...
    ...           ...
    trajectory   int32 4B 2
    state_names  (state) <U2 24B ...
    atNums       (atom) int64 96B ...
    delta_t      float64 8B ...
  * from         (statecomb) int64 24B 1 1 2
  * to           (statecomb) int64 24B 2 3 3
Data variables:
    energy       (state, time) float64 7kB 1.606 1.141 0.7258 ... 10.84 11.02
    nacs         (statecomb, atom, direction, time) float64 259kB 0.01678 ......
    atXYZ        (atom, direction, time) float64 86kB 0.2717 0.2719 ... -2.339
    phases       (state, time) float64 7kB -1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0
    e_kin        (time) float64 2kB 1.271 1.641 2.014 ... 5.779 5.627 5.466
    forces       (state, atom, direction, time) float64 259kB -0.08559 ... 0.0
    dip_perm     (state, direction, time) float64 22kB -0.2074 ... 0.08382
    dip_trans    (statecomb, direction, time) float64 22kB 0.7541 ... 0.3947
Attributes:
    delta_t:                0.5
    trajectory_input_path:  test_data/shnitsel/fixtures/butene_dynamic/data.nc
    trajectory_id:          1784668298, _is_multi_trajectory=False), Trajectory(_raw_dataset=<xarray.Dataset> Size: 672kB
Dimensions:      (state: 3, time: 300, statecomb: 3, atom: 12, direction: 3)
Coordinates: (12/17)
  * state        (state) int64 24B 1 2 3
  * time         (time) float64 2kB 0.0 0.5 1.0 1.5 ... 148.0 148.5 149.0 149.5
  * statecomb    (statecomb) object 24B MultiIndex
  * atom         (atom) int64 96B 0 1 2 3 4 5 6 7 8 9 10 11
  * direction    (direction) <U1 12B 'x' 'y' 'z'
    sdiag        (time) int64 2kB ...
    ...           ...
    trajectory   int32 4B 3
    state_names  (state) <U2 24B

Out of a dataset, a single variable can be selected similar to variables of any object in python:

In [4]:
frames.energy

<xarray.DataArray 'energy' (state: 3, frame: 36579)> Size: 878kB
[109737 values with dtype=float64]
Coordinates:
  * state        (state) int64 24B 1 2 3
  * frame        (frame) object 293kB MultiIndex
    sdiag        (frame) int64 293kB ...
    astate       (frame) int64 293kB ...
    state_names  (state) <U2 24B ...
  * atrajectory  (frame) int64 293kB 2 2 2 2 2 2 2 ... 316 316 316 316 316 316
  * time         (frame) float64 293kB 0.0 0.5 1.0 1.5 ... 76.5 77.0 77.5 78.0
Attributes:
    units:    eV
    unitdim:  Energy

Various data associated with a particular trajectory can be selected

In [5]:
# This selects general trajectory data from the metadata
frames.sel(trajectory=2)

Frames(_raw_dataset=<xarray.Dataset> Size: 83MB
Dimensions:      (state: 3, frame: 36579, statecomb: 3, atom: 12, direction: 3)
Coordinates: (12/19)
  * state        (state) int64 24B 1 2 3
  * frame        (frame) object 293kB MultiIndex
  * statecomb    (statecomb) object 24B MultiIndex
  * atom         (atom) int64 96B 0 1 2 3 4 5 6 7 8 9 10 11
  * direction    (direction) <U1 12B 'x' 'y' 'z'
    sdiag        (frame) int64 293kB ...
    ...           ...
    atNums       (atom) int64 96B ...
    delta_t      float64 8B ...
  * from         (statecomb) int64 24B 1 1 2
  * to           (statecomb) int64 24B 2 3 3
  * atrajectory  (frame) int64 293kB 2 2 2 2 2 2 2 ... 316 316 316 316 316 316
  * time         (frame) float64 293kB 0.0 0.5 1.0 1.5 ... 76.5 77.0 77.5 78.0
Data variables:
    energy       (state, frame) float64 878kB ...
    nacs         (statecomb, atom, direction, frame) float64 32MB ...
    atXYZ        (atom, direction, frame) float64 11MB 0.2717 0.2719 ... 1.624
    phases       (state, frame) float64 878kB ...
    e_kin        (frame) float64 293kB ...
    forces       (state, atom, direction, frame) float64 32MB ...
    dip_perm     (state, direction, frame) float64 3MB ...
    dip_trans    (statecomb, direction, frame) float64 3MB ...
Attributes:
    delta_t:                0.5
    trajectory_input_path:  test_data/shnitsel/fixtures/butene_dynamic/data.nc
    trajectory_id:          1784668298, _is_multi_trajectory=True)

Within the `frames` data, we may also be interested in singling out the data associated with the active trajectory:

In [7]:
frames.sel(atrajectory=2)

Trajectory(_raw_dataset=<xarray.Dataset> Size: 677kB
Dimensions:      (state: 3, time: 300, statecomb: 3, atom: 12, direction: 3,
                  trajectory: 156)
Coordinates: (12/18)
  * state        (state) int64 24B 1 2 3
  * time         (time) float64 2kB 0.0 0.5 1.0 1.5 ... 148.0 148.5 149.0 149.5
  * statecomb    (statecomb) object 24B MultiIndex
  * atom         (atom) int64 96B 0 1 2 3 4 5 6 7 8 9 10 11
  * direction    (direction) <U1 12B 'x' 'y' 'z'
  * trajectory   (trajectory) int32 624B 2 3 6 11 13 16 ... 310 311 313 314 316
    ...           ...
    state_names  (state) <U2 24B ...
    atNums       (atom) int64 96B ...
    delta_t      (trajectory) float64 1kB ...
  * from         (statecomb) int64 24B 1 1 2
  * to           (statecomb) int64 24B 2 3 3
    atrajectory  int64 8B 2
Data variables:
    energy       (state, time) float64 7kB ...
    nacs         (statecomb, atom, direction, time) float64 259kB ...
    atXYZ        (atom, direction, time) float64 86kB 0.2717 0.2719 ... -2.339
    phases       (state, time) float64 7kB ...
    e_kin        (time) float64 2kB ...
    forces       (state, atom, direction, time) float64 259kB ...
    dip_perm     (state, direction, time) float64 22kB ...
    dip_trans    (statecomb, direction, time) float64 22kB ...
Attributes:
    delta_t:                0.5
    trajectory_input_path:  test_data/shnitsel/fixtures/butene_dynamic/data.nc
    trajectory_id:          1784668298, _is_multi_trajectory=False)

Or if we want to select the metadata of a trajectory together with its active data:

In [6]:
frames.sel(atrajectory=2, trajectory=2)

Trajectory(_raw_dataset=<xarray.Dataset> Size: 672kB
Dimensions:      (state: 3, time: 300, statecomb: 3, atom: 12, direction: 3)
Coordinates: (12/18)
  * state        (state) int64 24B 1 2 3
  * time         (time) float64 2kB 0.0 0.5 1.0 1.5 ... 148.0 148.5 149.0 149.5
  * statecomb    (statecomb) object 24B MultiIndex
  * atom         (atom) int64 96B 0 1 2 3 4 5 6 7 8 9 10 11
  * direction    (direction) <U1 12B 'x' 'y' 'z'
    sdiag        (time) int64 2kB ...
    ...           ...
    state_names  (state) <U2 24B ...
    atNums       (atom) int64 96B ...
    delta_t      float64 8B ...
  * from         (statecomb) int64 24B 1 1 2
  * to           (statecomb) int64 24B 2 3 3
    atrajectory  int64 8B 2
Data variables:
    energy       (state, time) float64 7kB ...
    nacs         (statecomb, atom, direction, time) float64 259kB ...
    atXYZ        (atom, direction, time) float64 86kB 0.2717 0.2719 ... -2.339
    phases       (state, time) float64 7kB ...
    e_kin        (time) float64 2kB ...
    forces       (state, atom, direction, time) float64 259kB ...
    dip_perm     (state, direction, time) float64 22kB ...
    dip_trans    (statecomb, direction, time) float64 22kB ...
Attributes:
    delta_t:                0.5
    trajectory_input_path:  test_data/shnitsel/fixtures/butene_dynamic/data.nc
    trajectory_id:          1784668298, _is_multi_trajectory=False)

...or we can select a cross-section of all trajectories at a particular time

In [8]:
frames.sel(time=15.5)

Trajectory(_raw_dataset=<xarray.Dataset> Size: 354kB
Dimensions:      (state: 3, atrajectory: 156, statecomb: 3, atom: 12,
                  direction: 3, trajectory: 156)
Coordinates: (12/18)
  * state        (state) int64 24B 1 2 3
  * atrajectory  (atrajectory) int64 1kB 2 3 6 11 13 16 ... 310 311 313 314 316
  * statecomb    (statecomb) object 24B MultiIndex
  * atom         (atom) int64 96B 0 1 2 3 4 5 6 7 8 9 10 11
  * direction    (direction) <U1 12B 'x' 'y' 'z'
  * trajectory   (trajectory) int32 624B 2 3 6 11 13 16 ... 310 311 313 314 316
    ...           ...
    state_names  (state) <U2 24B ...
    atNums       (atom) int64 96B ...
    delta_t      (trajectory) float64 1kB ...
  * from         (statecomb) int64 24B 1 1 2
  * to           (statecomb) int64 24B 2 3 3
    time         float64 8B 15.5
Data variables:
    energy       (state, atrajectory) float64 4kB ...
    nacs         (statecomb, atom, direction, atrajectory) float64 135kB ...
    atXYZ        (atom, direction, atrajectory) float64 45kB 0.5114 ... 1.52
    phases       (state, atrajectory) float64 4kB ...
    e_kin        (atrajectory) float64 1kB ...
    forces       (state, atom, direction, atrajectory) float64 135kB ...
    dip_perm     (state, direction, atrajectory) float64 11kB ...
    dip_trans    (statecomb, direction, atrajectory) float64 11kB ...
Attributes:
    delta_t:                0.5
    trajectory_input_path:  test_data/shnitsel/fixtures/butene_dynamic/data.nc
    trajectory_id:          1784668298, _is_multi_trajectory=False)

Most operations can be vectorized, but due to the *stacked* format, rather than making 'trajectory' and 'time'
full dimensions, frames from all trajectories are laid along a single axis, using
a `MultiIndex` to keep track of which trajectory and time they belong to.

This precludes direct vectorized operations on trajectories or time.
However, we can group data within datasets and variables using the `.groupby()` method and process the grouped data
to perform operations on a per-trajectory basis.  

For example, to calculate the ratio of timesteps spent in $S_1$ vs in $S_0$:

In [10]:
import numpy as np
frames['astate'].groupby('atrajectory').map(lambda astate: np.sum(astate==2) / np.sum(astate==1))

<xarray.DataArray 'astate' (atrajectory: 156)> Size: 1kB
array([ 6.14285714,  0.78571429,  0.2987013 ,  0.11940299,  2.05263158,
        0.32258065,  0.91082803,  0.59848485,  2.94736842,  0.44508671,
        0.64835165,  0.42857143,  4.38235294,  0.875     ,  0.53409091,
        0.30434783,  4.26315789,  0.27962085,  7.        ,  0.39153439,
        1.35      ,  0.38167939,  0.93103448, 61.5       ,  0.67597765,
        0.74418605,  4.66666667,  5.80769231,  1.68686869,  1.225     ,
        1.36220472,  0.12359551,  5.38461538,  0.81818182,  2.09756098,
        0.16731518,  0.09489051,  3.16981132,  1.96774194,  0.10701107,
        0.68539326,  1.31818182,  0.85185185,  1.34375   ,  0.38248848,
        0.40594059,  0.8404908 ,  0.40206186,  0.30143541,  0.35135135,
        1.85714286,  0.98675497,  0.44230769,  0.2145749 , 37.75      ,
        1.2556391 ,  4.35294118,  0.40186916, 10.26315789,  2.51428571,
        0.88679245,  1.31578947,  2.03225806,         inf,  2.63888889,
        7.8       ,  1.98214286,  1.20588235,  0.51219512,  9.89473684,
        1.5       ,  4.39285714,  3.775     ,  0.75      ,  0.71428571,
        0.26153846,  1.27272727,  0.76470588,  0.18577075,  0.48484848,
        0.2987013 ,  0.38888889,  4.14285714,  1.98765432,         inf,
        0.25592417,         inf,  1.57291667,         inf,  1.5125    ,
        0.35748792,  3.20512821,  0.25      ,  6.74074074,  1.14492754,
               inf,  1.31404959, 10.11111111,  2.125     ,  0.59722222,
        1.45901639,  0.35746606,  0.32743363,  1.57142857,  1.14285714,
        1.09677419,  3.61538462,  1.08333333,  1.72727273,  0.15830116,
        0.35746606,  1.72727273,  4.73529412,         inf,  0.2       ,
        0.57068063, 20.14285714,  0.73410405,  1.5862069 ,  0.38372093,
        0.33928571,  0.45631068,  4.35135135,  1.34375   ,  0.2       ,
        0.70454545,  1.63333333,  4.76923077,  4.52      ,         inf,
        0.76470588,  1.5210084 ,  2.49180328,  2.475     ,  0.53846154,
        2.52173913,  0.25      ,  4.05882353,  0.37113402,  1.40186916,
        0.29310345,  0.8404908 ,  0.73410405,  1.43902439,  0.76470588,
        3.2173913 ,  6.5       ,  0.32743363,  3.10958904,  2.7826087 ,
        1.54237288,  0.70454545,  1.78823529,  0.62096774,  1.35849057,
        0.91463415])
Coordinates:
  * atrajectory  (atrajectory) int64 1kB 2 3 6 11 13 16 ... 310 311 313 314 316
Attributes:
    long_name:  active state (MCH)